In [2]:
import pymongo
from pymongo import MongoClient

import pandas as pd

from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file

import numpy as np

### mongo stuff

In [5]:
conn=MongoClient('mongodb://localhost:27017')
db=conn.local
collection=db.compensation_by_industry
col_grad_employment=db.graduates_employments
col_ULPT=db.ULPT

#### load stuff

In [ ]:
df=pd.read_csv("dataset/compensation-of-employees-by-industry-at-current-prices-annual.csv")
df=df.drop(['level_1'],axis=1)
df.rename(columns={'level_2':'industry','value':'compensation'}, inplace=True)
# print(df.to_dict('records'))
collection.insert_many(df.to_dict('records'))


In [ ]:
df2=pd.read_csv("dataset/graduate-employment-survey-ntu-nus-sit-smu-sutd.csv",encoding = "ISO-8859-1")
# print(df2)
col_grad_employment.insert_many(df2.to_dict('records'))

In [6]:
from bs4 import BeautifulSoup as bs
import requests

url="https://www.reddit.com/r/UnethicalLifeProTips/top/?t=all"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

response=requests.get(url,headers=headers)
content = response.text
# print(content)
soup=bs(content,"html.parser")

r=soup.find_all('h2',"fiq55l-0 kXfnwz")
ulpt= [R.text.strip() for R in r]
ulpt=[{"ulpt": R.replace("ULPT: ","")} for R in ulpt]
print (ulpt)

col_ULPT.insert_many(ulpt)

[{'ulpt': "Don't ask your girl where she wants to eat. Instead, tell her to guess where you're taking her to eat. Then take her to the first guess"}, {'ulpt': 'If your college website uses \'TurnItIn\' to check for plagiarism, simply add "quotes" at the beggining and end of your entire paper in white font and it will interpret this as one long citation, giving you a 0% match.'}, {'ulpt': 'Working the last shift at a restaurant? Have a friend call in an order for pickup and not show up, then just take your free food home.'}, {'ulpt': "Save your dead light bulbs; whenever you are at a hotel, swap your bulbs for your room's bulbs"}, {'ulpt': "If your man asks you to guess where he's taking you to eat, guess the most expensive place you've ever wanted to eat."}, {'ulpt': 'Need some money? Over the legal alcohol purchasing age? Go to a high school party and offer to buy booze. Collect money from all the kids and never come back'}, {'ulpt': 'If you’re a guy and your wife/girlfriend is super 

trying bokeh (nahhhhhhhhhhh)

In [ ]:
from bokeh.palettes import YlGnBu,brewer
import pygal

with conn as client:
    df_compensation_industry=pd.DataFrame(list(collection.find()))
df_compensation_industry=df_compensation_industry.drop(['_id'],axis=1)
print(df_compensation_industry.info())


x=df_compensation_industry.year.unique()
xRange=np.char.mod('%d',x)
yRange=df_compensation_industry.industry.unique()
yRange=np.array(yRange)

data=df_compensation_industry.to_dict('records')
# print(data)

output_file("graph1.html")
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"
plt1=figure(plot_width=500, plot_height=500,tools=TOOLS)
plt1.title.text="Compensation of Industry by Year"

for key,color in zip(yRange,brewer['Spectral'][yRange.size]):
    y= df_compensation_industry[:][df_compensation_industry.industry==key].compensation
    plt1.line(x,y,line_width=2,color=color,legend=key)
    
plt1.legend.location = "top_left"
plt1.legend.click_policy="hide"


show(plt1)